In [1]:
import matplotlib.font_manager as font_manager
from matplotlib.lines import Line2D
import matplotlib as mpl
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
from scipy.stats import bootstrap
import statsmodels.api as sm
import statsmodels.formula.api as smf

params = {
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "font.size": 12,
    "legend.fontsize": 12,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "text.usetex": False,
}

mpl.rcParams.update(params)


import warnings

warnings.filterwarnings("ignore")


c:\Users\nadal\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
df_time_series = pd.read_csv(
    "./data/df_timeseries_en.tsv.gz", compression="infer", sep="\t"
)
df_time_series["datetime"] = pd.to_datetime(df_time_series["datetime"])

df_channels = pd.read_csv("./data/df_channels_en.tsv.gz", compression="infer", sep="\t")
df_channels["join_date"] = pd.to_datetime(df_channels["join_date"])
video_metadatas = pd.read_feather(
    "./data/yt_metadata_helper.feather",
    columns=[
        "categories",
        "upload_date",
        "duration",
        "like_count",
        "view_count",
        "channel_id",
    ],
)


In [ ]:
# round the subs value, easier to consider 1 sub as a person and not half of a person...
df_time_series.subs = df_time_series.subs.round(0)


In [ ]:
df_time_series.head()


In [ ]:
df_channels.head()


In [ ]:
video_metadatas.head()


# Check length of videos

An idea would be to use cut to match length of videos into discrete intervals 

In [ ]:
video_metadatas.duration.describe()


In [ ]:
# To make it easier to grasp the length of videos, we will convert the lengths to minutes at first...
video_metadatas["duration_min"] = video_metadatas["duration"] / 60
video_metadatas.duration_min.describe()


In [ ]:
plt.boxplot(video_metadatas["duration_min"], showfliers=False, vert=False)
plt.title("Duration of Videos in Minutes")
plt.xlabel("Duration in Minutes")


In [ ]:
duration_min_cumul = plt.hist(
    video_metadatas.duration_min, bins=100, log=True, cumulative=-1, histtype="step"
)
plt.title("Histogram of Duration for YouTube Videos (cumulative)")
plt.ylabel("# of Videos (in log scale)")
plt.xlabel("Duration in Minutes")
plt.xticks(rotation=30, ha="right")
plt.show()


In [ ]:
plt.loglog(duration_min_cumul[1][1:], duration_min_cumul[0])
plt.title("Histogram of Duration for YouTube Videos (cumulative)")
plt.ylabel("# of Videos (in log scale)")
plt.xlabel("Duration")
plt.xticks(rotation=30, ha="right")
plt.show()


In [ ]:
# long to compute

# bucket_durations = pd.cut(video_metadatas['duration'], bins=100)
# print(type(bucket_durations))


---

# Check frequency of videos

In [ ]:
video_count = (
    video_metadatas.groupby(pd.Grouper(key="upload_date", freq="W")).count().channel_id
)


In [ ]:
plt.plot(video_count, color="#7570b3", ls="--", label="\% videos uploaded")
plt.yscale("log")
plt.ylabel("Number of Videos")
plt.xlabel("Time")
plt.title("Number of videos updated each week")


In [ ]:
# now we will determine how many videos are uploaded to YouTube by each channel by year
video_metadatas["yearkNumber"] = video_metadatas["upload_date"].dt.year
vd_cnt_by_id_yr = (
    video_metadatas.groupby(["channel_id", "yearkNumber"])
    .count().categories.unstack().reset_index()
)
vd_cnt_by_id_yr.columns.name = None

In [ ]:
# we now remove the channels that uploaded less than 12 videos throughout the study period
vd_cnt_by_id_yr_filtered = vd_cnt_by_id_yr[vd_cnt_by_id_yr.sum(axis=1)>12]

In [ ]:
vd_cnt_by_id_yr_filtered.head()

In [ ]:
vd_cnt_by_id_yr_filtered_summary = vd_cnt_by_id_yr_filtered.describe()

In [ ]:
vd_cnt_by_id_yr_filtered_summary

In [ ]:
plt.fill_between(x=list(vd_cnt_by_id_yr_filtered_summary.columns.values),
                y1=list(vd_cnt_by_id_yr_filtered_summary.loc["25%",:]),
                y2=list(vd_cnt_by_id_yr_filtered_summary.loc["75%",:]), 
                 alpha = 0.5, color = 'gray')
plt.plot(vd_cnt_by_id_yr_filtered_summary.loc["50%",:], color = 'black')

plt.xlabel('Year')
plt.ylabel('Yearly Upload Frequency')
plt.title('The quantiles of yearly video upload frequency')
# here we choose quantiles, since mean would be significantly affected by the extremely high yearly upload rates

---

# How long to reach 1M subscribers 

Could be interesting to take all the videos with 10K, 20K, 30K, ... and see the evolution to reach 100K for example. And compare different evolution : from 50K to 100K or from 500K to 600K, etc.. which one is the fastest, easiest ?  

In [ ]:
def get_df_channel_ascension(start_subs, end_subs):
    """finds the channels in the time series that have less than the start_subs and more than end_subs.
    It means that it begins with less than start_subs and have now at least end_subs

    Args:
        start_subs (float): low_threshold
        end_subs (float): high_threshold

    Returns:
        dataframe containing all the channels with evolution from starts_subs to end_subs
    """
    under_start_subs = df_time_series[df_time_series["subs"] < start_subs]
    more_end_subs = df_time_series[df_time_series["subs"] > end_subs]
    channel_start_to_end = df_time_series[
        df_time_series.channel.isin(under_start_subs.channel)
        & df_time_series.channel.isin(more_end_subs.channel)
    ]
    return channel_start_to_end


In [ ]:
def get_mean_time_to_reach_X_subs(df, lower_bound, upper_bound):
    time_to_reach_X_subs = df.groupby("channel").apply(
        lambda group: pd.Series(
            {
                "from_zero_to_hero_duration": group[group.subs > upper_bound][
                    "datetime"
                ].iloc[0]
                - group[group.subs < lower_bound]["datetime"].iloc[-1]
            }
        )
    )
    return time_to_reach_X_subs


In [ ]:
channel_10K_to_1M = get_df_channel_ascension(10_000, 1_000_000)
print(
    "We have {} channels that begins with 10K subs and reach at least 1M".format(
        channel_10K_to_1M.channel.nunique()
    )
)


In [ ]:
first_channel = channel_10K_to_1M.iloc[0]
test = channel_10K_to_1M[channel_10K_to_1M["channel"] == first_channel.channel]
# test = test[(test['datetime'] > np.datetime64('2018-07-20')) & (test['datetime'] < np.datetime64('2019-01-20'))]
test.plot(x="datetime", y="subs")
print("There are {} weeks for this channel".format(test.shape[0]))
print(
    "We should have approximately the same value : \n{} and {}".format(
        7 * test.shape[0], test.iloc[-1].datetime - test.iloc[0].datetime
    )
)


In [ ]:
# get the time it takes to go from less than 10K to 1M
time_to_reach_1M = get_mean_time_to_reach_X_subs(channel_10K_to_1M, 10_000, 1_000_000)
time_to_reach_1M.head()


In [ ]:
print(
    "In average YTbers take {} days to reach 1M of subscribers".format(
        time_to_reach_1M["from_zero_to_hero_duration"].mean().days
    )
)


It would be nice to bootstrap this result to see with interval of confidence the time taken by Youtuber to reach 1M

In [ ]:
data = time_to_reach_1M["from_zero_to_hero_duration"].apply(
    lambda delta_time: delta_time.days
)


In [ ]:
rng = np.random.default_rng()
conf_interval_reach_1M = bootstrap(
    (data,),
    np.mean,
    confidence_level=0.95,
    random_state=rng,
).confidence_interval
conf_interval_reach_1M


In [ ]:
print(
    "The 95% interval of confidence, the time taken to reach 1M (from 10K) is [{}, {}]".format(
        conf_interval_reach_1M.low, conf_interval_reach_1M.high
    )
)


---

# Check the number of views (maybe after we could add likes, dislike) given a certain categories and date of videos

For this task lets analyze and after ploting and getting some intuition, use logistic regression to 'predict' the nb of views given category and date 

In [ ]:
video_metadatas.head()
video_metadatas.shape


In [ ]:
# delete the rows with empty categories
videos_with_cat = video_metadatas
videos_with_cat.categories.replace("", np.nan, inplace=True)
videos_with_cat.dropna(subset=["categories"], inplace=True)
videos_with_cat.shape


In [ ]:
encoded_cat = pd.get_dummies(videos_with_cat.categories)
video_metadatas_encoded = videos_with_cat.join(encoded_cat)


In [ ]:
video_metadatas_encoded["year"] = video_metadatas_encoded["upload_date"].dt.year


In [ ]:
video_metadatas_encoded.head()


In [ ]:
video_metadatas_encoded.columns

In [ ]:
# Wenxiu Comments: we probably should change the naming of columns - at least on my laptop the ones with & are not identified
model = smf.ols(
    formula='view_count ~ year + C(Autos & Vehicles) + C(Comedy) + C(Education) + C(Entertainment) + C(Film & Animation) + \
                C(Gaming) + C(Movies) + C(Music) + C(News & Politics) + C(Nonprofits & Activism) + \
                C(People & Blogs) + C(Pets & Animals) + C(Science & Technology) + C(Shows) + C(Sports) + C(Travel & Events)',
    data=video_metadatas_encoded
)

res = model.fit()
print(res.summary())


---

In [ ]:
top_ranked_channels = df_channels[df_channels.subscriber_rank_sb < 100]
top_ranked_channels.head()


In [ ]:
date = top_ranked_channels["join_date"].apply(lambda d: d.to_pydatetime().year)
np.mean(date, axis=0)


In [ ]:
oldest_video = df_channels["join_date"].apply(lambda d: d.to_pydatetime().year)
oldest_video.nsmallest(5)


In [ ]:
channels_with_largest_subscribers = df_channels.nlargest(53, "subscribers_cc")
channels_with_largest_subscribers.sample(7)


In [ ]:
dfs = []

---

# Most common tags per category

(Paul) I will start this part when I find how to create videos metadata dataset with tags (& title)

In [2]:
tags = pd.read_feather(
    "./data/yt_metadata_tags_helper.feather"
)

In [3]:
tags.shape
tags.head()

,categories,tags
0,Film & Animation,"lego city,lego police,lego city police,lego ci..."
1,Film & Animation,"Lego superheroes,lego hulk,hulk smash,lego mar..."
2,Film & Animation,"lego city,lego police,lego city police,lego fi..."
3,Film & Animation,"Lego harry potter,new harry potter,harry potte..."
4,Film & Animation,"lego city,lego police,lego city police,lego fi..."
